# Property calculation with SAFT-VR-Mie EoS

First, it is necessary to import the ``component`` class and the SAFT-VR-Mie equation of state (``saftvrmie``).

In [1]:
import numpy as np
from sgtpy import component, saftvrmie

First, a component is defined with the ``component`` class function, then the ``eos`` object is created with the component and the ``saftvrmie`` function. 

The ``eos`` object includes the methods to evaluate properties from the equation of state, such as densities, pressure, fugacity coefficients, chemical potential and some thermal derived properties (residual entropy, residual enthalpy, residual heat capacities and speed of sound).

**warning:** thermal derived properties are computed with numerical derivatives using $O(h^4)$ approximation.

In the case of coarse-grained non-associating fluids, the ``eos`` object allows computing the [influence parameter for SGT](https://aiche.onlinelibrary.wiley.com/doi/full/10.1002/aic.15190). This is done with the following correlation:

$$ \sqrt{\frac{c_{ii}}{N_{av}^2 \epsilon_{ii} \sigma_{ii}^5}} = m_s \left[0.12008 + 2.21979 \alpha_i \right]$$

$$ \alpha_i = \left[ \frac{\lambda_r}{\lambda_r - \lambda_a} \left( \frac{\lambda_r}{\lambda_a}\right)^{\frac{\lambda_a}{\lambda_r - \lambda_a}}\right] \left[\frac{1}{\lambda_a - 3} - \frac{1}{\lambda_r - 3} \right]$$

The ``eos.cii_correlation`` method is shown below for methane.

In [2]:
methane = component('methane', ms = 1.0, sigma = 3.752 , eps = 170.75,
                    lambda_r = 16.39, lambda_a = 6.)
eos = saftvrmie(methane)
eos.cii_correlation(overwrite=True)

array([1.92075094e-20])

In the following examples, self-associating 4C water will be used alongside SAFT-VR-Mie EoS.

In [3]:
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6., eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], Mw = 18.01528, cii = 1.5371939421515458e-20)
eos = saftvrmie(water)

When you run ``saftvrmie(component)``, SGTPy will attempt to compute the critical point of the fluid. If the procedure is successful you can check the computed values with the ``eos.Tc``, ``eos.Pc`` and ``eos.rhoc`` attributes. The bool ``eos.critical`` indicates if the computation was successful. 

In [4]:
print('Critical point calculation success: ', eos.critical)
if eos.critical:
    print('Critical temperature:', eos.Tc, 'K')
    print('Critical pressure:', eos.Pc, 'Pa')
    print('Critical density:', eos.rhoc, 'mol/m^3')
    


Critical point calculation success:  True
Critical temperature: 694.7496645164509 K
Critical pressure: 33146202.40897208 Pa
Critical density: 17706.26360567451 mol/m^3


If the critical point calculation is not successful, you can try to compute the critical point of the fluid using the ``eos.get_critical`` method. The method returns the critical temperature [K], critical pressure [Pa] and critical density [mol/m^3]. You can supply better initial values to start the non-linear solver.

If you set the parameter ``overwrite=True``, the results from this method will overwrite the previous computed critical point at initialization.

In [5]:
# get_critical providing initial guesses
Tc0 = 690.  # K
rhoc0 = 17500.  # mol/m3

eos.get_critical(Tc0, rhoc0, overwrite=True)

(694.7496645165976, 33146202.409016237, 17706.265339522575)

The density of the fluid is computed with the ``eos.density`` method. It requires the temperature (K), pressure (Pa) and the aggregation state (``'L'`` for liquid phase or ``'V'`` for vapor phase).

When no initial guess has been provided. Topliss's method is used to initialize the density calculation.

In [6]:
T = 300.  # K
P = 1e5  # Pa
# computed density in mol/m3
eos.density(T, P, 'L'), eos.density(T, P, 'V')

(56938.97048119237, 44.96277234775467)

Optionally, you can provide an initial guess to start computing the density. This is done with the ``rho0`` parameter.

In this case, Newton's method is used to solve the density.

In [7]:
T = 300.  # K
P = 1e5  # Pa
# computed density in mol/m3
rho0 = 0.95*56938.97048119255 # mol/m3
eos.density(T, P, 'L', rho0=rho0)

56938.97048119243

Similarly, the pressure of the fluid can be computed at given molar density (mol/m3) and temperature (K) using the ``eos.pressure`` method. 

In [8]:
T = 300.  # K
rhov = 44.962772347754836  # mol/m3
rhol = 56938.970481192526  # mol/m3
#computed pressure in Pa
eos.pressure(rhov, T), eos.pressure(rhol, T)

(99999.99999658794, 100000.00000259202)

For pure fluids, the ``eos.psat`` method allows computing the saturation pressure at given temperature. This method returns the equilibrium pressure and molar volumes of the liquid and vapor phases. Similarly, the ``eos.tsat`` method allows computing the saturation temperature at given pressure.

The phase equilibria can be verified through fugacity coefficients using the ``eos.logfug`` method or by using chemical potentials with the ``eos.muad`` method. The chemical potentials require that dimensionless density and temperature.

In [9]:
T = 300.  # K
P0 = 1e3  # Pa
# equilibrium pressure (Pa), liquid volume (m3/mol), vapor volume (m3/mol)
eos.psat(T, P0=P0)

(3640.841209122666, 1.756342718267362e-05, 0.6824190076059875)

In [10]:
P = 1e3  # Pa
T0 = 300.  # K
# equilibrium temperature (K), liquid volume (m3/mol), vapor volume (m3/mol)
eos.tsat(P, T0=T0)

(279.2064179224693, 1.7318625791835306e-05, 2.3171333573153374)

Alternatively, if you don't supply an initial guess for the pressure and the critical point of the fluid was correctly computed at initialization, the pressure will be automatically initiated using the [zero-pressure o Pmin/Pmax algorithm](https://www.sciencedirect.com/science/article/pii/S0098135497000161). 

In [11]:
# computing VLE without initial guess
# equilibrium pressure (Pa), liquid volume (m3/mol), vapor volume (m3/mol)
eos.psat(T)

(3640.84120912265, 1.7563427182673632e-05, 0.682419007616849)

You can check that the phase equilibria was computed correctly, verifying either the chemical potential or fugacity coefficients of the phases.

In [12]:
Psat = 3640.841209122654  # Pa
vl = 1.756342718267362e-05  # m3/mol
vv = 0.6824190076059896  # m3/mol

# checking chemical potentials
np.allclose(eos.muad(1/vl, T) , eos.muad(1/vv, T))

True

In [13]:
# checking fugacity coefficients
np.allclose(eos.logfug(T, Psat, 'L', v0=vl)[0], eos.logfug(T, Psat, 'V', v0=vv)[0])

True

The ``eos`` object also includes the calculation of some thermal derived properties such as residual entropy (``eos.EntropyR``), residual enthalpy (``eos.EnthalpyR``), residual isochoric heat capacity (``eos.CvR``), , residual isobaric heat capacity (``eos.CpR``).

For the speed of sound calculation (``eos.speed_sound``) the ideal gas heat capacities are required, in the example the isochoric and isobaric ideal gas contributions are set to $3R/2$ and $5R/2$, respectively. Better values of ideal gas heat capacities contribution can be found from empirical correlations, such as the provided by DIPPR 801.

In [14]:
# vaporization entropy in J/mol K
Sl = eos.EntropyR(T, Psat, 'L', v0=vl)
Sv = eos.EntropyR(T, Psat, 'V', v0=vv)
Svap = Sv - Sl

# vaporization enthalpy in J/mol
Hl = eos.EnthalpyR(T, Psat, 'L')
Hv = eos.EnthalpyR(T, Psat, 'V')
Hvap = Hv - Hl


# isochoric and isobaric residual heats capacities in J / mol K
cvr = eos.CvR(1/vl, T)
cpr = eos.CpR(T, Psat, 'L')


# ideal gas heat capacities, better values can be obtained with DIPPR 801 correlations
r = 8.314  # J / mol K
CvId = 3*r/2
CpId = 5*r/2
w = eos.speed_sound(T, Psat, 'V', v0=vl, CvId=CvId, CpId=CpId)

print('Vaporization Entropy : ', Svap, 'J / mol K')
print('Vaporization Enthalpy : ', Hvap, 'J / mol')
print('Residual isochoric heat capacity : ', cvr, 'J / mol K')
print('Residual isobaric heat capacity : ', cpr, 'J / mol K')
print('Speed of sound : ', w, 'm / s')

Vaporization Entropy :  142.8199794927401 J / mol K
Vaporization Enthalpy :  42845.99384783339 J / mol
Residual isochoric heat capacity :  30.83559197746939 J / mol K
Residual isobaric heat capacity :  28.20967588845166 J / mol K
Speed of sound :  1563.6575055525836 m / s


To get better values of the speed of sound you can provide correlated values for the ideal gas heat capacities.

In [15]:
# ideal heat capacity from DIPPR 801.

k1=33363
k2=26790
k3=2610.5
k4=8896
k5=1169

CpId = k1 + k2 * ((k3/T) /np.sinh(k3/T))**2
CpId += k4 * ((k5/T) /np.cosh(k5/T))**2
CpId /= 1000.

CvId = CpId - r

# better value for speed of sound (m/s)
eos.speed_sound(T, Psat, 'L', v0=vl, CvId=CvId, CpId=CpId)

1542.8100392829724

---
For further information about each method check out the documentation running: ``eos.function?``